<a href="https://colab.research.google.com/github/ashaghaghi20/legal-asist-bot/blob/main/Lawbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %% [markdown]
# ## نصب پیش‌نیازها
# اجرای این سلول ممکن است چند دقیقه زمان ببرد.

# %%
# Install necessary libraries including nest-asyncio and upgrade conflicting ones for dependency resolution
!pip install --upgrade python-telegram-bot nest-asyncio whisper-openai pytube openai python-dotenv sentence-transformers transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.0/731.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.9/964.9 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:
      Successfully uninstalled transformers-4.56.2


In [ ]:
# Removed redundant cell execution

## انجام تست‌های جامع

### Subtask:
ربات هم‌اکنون فعال است. عملکرد آن را به طور کامل تست کنید. این شامل ارسال پیام‌های صوتی برای تست قابلیت تبدیل صوت به متن، ارسال متون مختلف (حقوقی و غیرحقوقی) برای تست قابلیت تحلیل متن، بررسی پاسخ ربات به انتخاب‌های منو، و تست سناریوهای مربوط به عضویت در کانال و سهمیه استفاده است. نتایج تست را ثبت و هرگونه مشکل باقی‌مانده را شناسایی کنید.

In [ ]:
# -*- coding: utf-8 -*-
# Install necessary libraries and upgrade conflicting ones for dependency resolution
!pip install --upgrade python-telegram-bot nest-asyncio whisper-openai pytube openai python-dotenv sentence-transformers transformers torch

import os
import logging
import sqlite3
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import whisper
import tempfile
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
import asyncio
import nest_asyncio # Import nest_asyncio
# import threading # Import threading - Removed threading as it conflicts with asyncio in this context

# Apply nest_asyncio to allow nested event loops, necessary for running asyncio in environments like Colab
nest_asyncio.apply()


# import torchaudio # Uncomment this line if you want to use torchaudio for audio loading

# تنظیمات لاگ‌گیری
logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO)
logger = logging.getLogger(__name__)

# توکن ربات و آیدی کانال — این مقادیر را جایگزین کرده‌اید
BOT_TOKEN = os.environ.get("TELEGRAM_BOT_TOKEN", "7693531934:AAEwCi1itefgAgdVZz_gvGLUc0DfbgkS6Tc") # Corrected variable name
SPONSOR_CHANNEL = os.environ.get("SPONSOR_CHANNEL", "@Radio_Zhelofen") # Corrected variable name

# --- پایگاه داده برای مدیریت کاربران و سهمیه ---
DB_NAME = 'users.db'

def init_db():
    conn = None
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS users
                     (user_id INTEGER PRIMARY KEY,
                      usage_count INTEGER DEFAULT 0,
                      referral_count INTEGER DEFAULT 0)''')
        conn.commit()
        logger.info("Database initialized.")
    except sqlite3.Error as e:
        logger.error(f"Error initializing database: {e}")
    finally:
        if conn:
            conn.close()


def get_user_data(user_id):
    conn = None
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        c.execute("SELECT usage_count, referral_count FROM users WHERE user_id=?", (user_id,))
        result = c.fetchone()
        if result:
            return {'usage_count': result[0], 'referral_count': result[1]}
        add_user(user_id) # Ensure user exists before returning default
        return {'usage_count': 0, 'referral_count': 0}
    except sqlite3.Error as e:
        logger.error(f"Error retrieving user data for user {user_id}: {e}")
        return {'usage_count': 0, 'referral_count': 0} # Return default in case of error
    finally:
        if conn:
            conn.close()

def add_user(user_id):
     conn = None
     try:
         conn = sqlite3.connect(DB_NAME)
         c = conn.cursor()
         c.execute("INSERT OR IGNORE INTO users (user_id, usage_count, referral_count) VALUES (?, 0, 0)", (user_id,))
         conn.commit()
         logger.info(f"User {user_id} added to DB if not exists.")
     except sqlite3.Error as e:
         logger.error(f"Error adding user {user_id} to DB: {e}")
     finally:
         if conn:
             conn.close()


def increment_usage(user_id):
    conn = None
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        c.execute("UPDATE users SET usage_count = usage_count + 1 WHERE user_id = ?", (user_id,))
        conn.commit()
        logger.info(f"Usage count incremented for user {user_id}.")
    except sqlite3.Error as e:
        logger.error(f"Error incrementing usage for user {user_id}: {e}")
    finally:
        if conn:
            conn.close()

def add_referral(referrer_id):
    conn = None
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        c.execute("UPDATE users SET referral_count = referral_count + 1 WHERE user_id = ?", (referrer_id,))
        conn.commit()
        logger.info(f"Referral count incremented for user {referrer_id}.")
    except sqlite3.Error as e:
        logger.error(f"Error adding referral for user {referrer_id}: {e}")
    finally:
        if conn:
            conn.close()

# --- بررسی عضویت در کانال ---
async def check_channel_membership(user_id, context: ContextTypes.DEFAULT_TYPE):
    try:
        member = await context.bot.get_chat_member(chat_id=SPONSOR_CHANNEL, user_id=user_id)
        return member.status in ['member', 'administrator', 'creator']
    except Exception as e:
        logger.error(f"خطا در بررسی عضویت: {e}")
        return False


# Load the Whisper model once at the beginning
whisper_model = None
try:
    # Consider loading a smaller model if memory is an issue, e.g., "base.en" or "small"
    # Also consider specifying device="cuda" if a GPU is available and you have torch with CUDA
    whisper_model = whisper.load_model("large", device="cpu") # Changed model to "large" and device to "cpu"
    logger.info("Whisper model loaded successfully.")
except Exception as e:
    logger.error(f"Error loading Whisper model: {e}")


# Load a text analysis model using Hugging Face pipeline
text_analyzer = None
try:
    # Using a general-purpose text classification model for Persian as a placeholder.
    # Finding a specific legal text analysis model for Persian in open-source is difficult.
    # This model is a general Persian text classification model. Its suitability for legal analysis is limited.
    # Replace with a more suitable model if one is found.
    # Using a valid Persian text classification model from Hugging Face
    model_name = "HooshvareLab/bert-fa-zwnj-base" # Corrected model name - using a general purpose Persian BERT model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    # The pipeline task might need adjustment depending on the specific model's intended use (e.g., sentiment-analysis, sequence-classification)
    # For a general BERT model, 'feature-extraction' or 'fill-mask' might be more appropriate if not fine-tuned for a specific task.
    # Assuming a classification task for now, but this might need refinement based on the desired legal analysis.
    text_analyzer = pipeline("text-classification", model=model, tokenizer=tokenizer)
    logger.info("Text analysis model loaded successfully.")
except Exception as e:
    logger.error(f"Error loading text analysis model: {e}")


async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """دستور /start — بررسی عضویت در کانال و نمایش منو"""
    user_id = update.effective_user.id

    add_user(user_id) # Ensure user exists for quota checks

    if not await check_channel_membership(user_id, context):
        await update.message.reply_text(
            f"⚠️ برای استفاده از ربات، لازم است ابتدا در کانال اسپانسر ما عضو شوید:\n\n"
            f"🌐 {SPONSOR_CHANNEL}\n\n"
            "پس از عضویت، مجدداً دستور /start را ارسال کنید."
        )
        return

    # If user is a member, show the main menu
    keyboard = [
        ["🎤 تبدیل صوت به متن", "📝 تحلیل حقوقی متن"],
        ["📄 ساخت قرارداد", "ℹ️ راهنما"]
    ]
    reply_markup = {"keyboard": keyboard, "resize_keyboard": True}
    await update.message.reply_text(
        "🔰 به ربات هوشمند حقوقی خوش آمدید!\n\n"
        "لطفاً یکی از گزینه‌های زیر را انتخاب کنید:",
        reply_markup=reply_markup
    )

async def handle_voice(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """مدیریت پیام‌های صوتی و تبدیل به متن"""
    user_id = update.effective_user.id

    # Check membership first
    if not await check_channel_membership(user_id, context):
         await update.message.reply_text(
            f"⚠️ برای استفاده از ربات، لازم است ابتدا در کانال اسپانسر ما عضو شوید:\n\n"
            f"🌐 {SPONSOR_CHANNEL}\n\n"
            "پس از عضویت، مجدداً دستور /start را ارسال کنید."
        )
         return

    # Then check usage quota
    user_data = get_user_data(user_id)
    FREE_USAGE_LIMIT = 5
    REFERRAL_REQUIRED = 5

    if user_data['usage_count'] >= FREE_USAGE_LIMIT and user_data['referral_count'] < REFERRAL_REQUIRED:
        await update.message.reply_text(
            f"❌ سهمیه رایگان {FREE_USAGE_LIMIT} تایی شما به پایان رسید.\n"
            f"برای ادامه استفاده، باید {REFERRAL_REQUIRED} کاربر را معرفی کنید.\n"
            "برای دریافت لینک دعوت خود، با مدیر تماس بگیرید." # Simplified message as referral link generation is not fully implemented
        )
        return

    if whisper_model is None:
        await update.message.reply_text("❌ متأسفانه مدل تبدیل صوت به متن بارگذاری نشده است. لطفاً به مدیر ربات اطلاع دهید.")
        return

    await update.message.reply_text("✅ پیام صوتی شما دریافت شد. در حال پردازش...")

    audio_path = None # Initialize audio_path to None
    try:
        voice_file = await update.message.voice.get_file()

        with tempfile.NamedTemporaryFile(delete=False, suffix='.ogg') as tmp_file:
            audio_path = tmp_file.name

        await voice_file.download_to_drive(audio_path)

        # Added error handling around transcription
        try:
            # Consider adding options like fp16=False if not using GPU or seeing issues
            result = whisper_model.transcribe(audio_path, language='fa')
            transcribed_text = result.get("text", "خطا در استخراج متن")
        except Exception as transcribe_error:
            logger.error(f"Error during Whisper transcription: {transcribe_error}")
            transcribed_text = "❌ خطایی در تبدیل پیام صوتی به متن رخ داد."


        increment_usage(user_id)
        user_data = get_user_data(user_id)
        remaining_uses = max(0, FREE_USAGE_LIMIT - user_data['usage_count'])

        response_text = f"📝 متن استخراج شده:\n\n{transcribed_text}\n\n"
        if remaining_uses > 0:
             response_text += f"🔢 شما {remaining_uses} استفاده رایگان دیگر دارید."
        else:
             response_text += f"💡 سهمیه رایگان شما به پایان رسید. برای ادامه استفاده، لطفا {REFERRAL_REQUIRED} کاربر را معرفی کنید."

        await update.message.reply_text(response_text)

    except Exception as e:
        logger.error(f"Error handling voice message: {e}")
        await update.message.reply_text("❌ خطایی در پردازش پیام صوتی شما رخ داد.")
    finally:
        # Ensure the temporary file is deleted
        if audio_path and os.path.exists(audio_path):
            try:
                os.unlink(audio_path)
                logger.info(f"Temporary file deleted: {audio_path}")
            except Exception as cleanup_error:
                logger.error(f"Error deleting temporary file {audio_path}: {cleanup_error}")



async def handle_text(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """مدیریت پیام‌های متنی و تحلیل حقوقی"""
    user_id = update.effective_user.id
    user_text = update.message.text

    # Check membership first
    if not await check_channel_membership(user_id, context):
         await update.message.reply_text(
            f"⚠️ برای استفاده از ربات، لازم است ابتدا در کانال اسپانسر ما عضو شوید:\n\n"
            f"🌐 {SPONSOR_CHANNEL}\n\n"
            "پس از عضویت، مجدداً دستور /start را ارسال کنید."
        )
         return


    if user_text == "🎤 تبدیل صوت به متن":
        await update.message.reply_text("لطفاً پیام صوتی خود را ارسال کنید.")
    elif user_text == "📝 تحلیل حقوقی متن":
        await update.message.reply_text("لطفاً متن حقوقی خود را برای تحلیل ارسال کنید.")
    elif user_text == "📄 ساخت قرارداد":
        await update.message.reply_text("این قابلیت به زودی فعال خواهد شد.")
    elif user_text == "ℹ️ راهنما":
         await update.message.reply_text(
             "📚 **راهنمای استفاده از ربات:**\n\n"
             "• برای تبدیل صوت به متن، گزینه '🎤 تبدیل صوت به متن' را انتخاب کرده و سپس پیام صوتی خود را ارسال کنید.\n"
             "• برای تحلیل متن حقوقی، گزینه '📝 تحلیل حقوقی متن' را انتخاب کرده و سپس متن مورد نظر را ارسال کنید.\n"
             "• قابلیت '📄 ساخت قرارداد' به زودی اضافه خواهد شد.\n"
             "• برای بازگشت به منوی اصلی، دستور /start را ارسال کنید."
         )
    elif text_analyzer is None:
        await update.message.reply_text("❌ متأسفانه مدل تحلیل متن بارگذاری نشده است. لطفاً به مدیر ربات اطلاع دهید.")
    else:
        # Apply usage quota check for text analysis
        user_data = get_user_data(user_id)
        FREE_USAGE_LIMIT = 5
        REFERRAL_REQUIRED = 5

        if user_data['usage_count'] >= FREE_USAGE_LIMIT and user_data['referral_count'] < REFERRAL_REQUIRED:
            await update.message.reply_text(
                f"❌ سهمیه رایگان {FREE_USAGE_LIMIT} تایی شما به پایان رسید.\n"
                f"برای ادامه استفاده، باید {REFERRAL_REQUIRED} کاربر را معرفی کنید.\n"
                "برای دریافت لینک دعوت خود، با مدیر تماس بگیرید."
            )
            return


        await update.message.reply_text("✅ متن شما دریافت شد. در حال تحلیل حقوقی...")
        formatted_result = "⚠️ تحلیل متن با مشکلی مواجه شد یا نتیجه‌ای در دسترس نیست." # Initialize formatted_result
        try:
            # Added error handling around text analysis
            try:
                analysis_result = text_analyzer(user_text)
                if analysis_result and isinstance(analysis_result, list) and len(analysis_result) > 0:
                     formatted_result = f"📊 نتیجه تحلیل:\n\n"
                     # The output format depends on the specific model.
                     # For a classification model, it might be a list of labels and scores.
                     # Adjust this formatting based on the actual output of the chosen Persian model.
                     # Example formatting assuming a list of dicts with 'label' and 'score':
                     for item in analysis_result:
                          # Need to handle potential list of lists or other structures depending on the model
                          if isinstance(item, list):
                              for sub_item in item:
                                   formatted_result += f"- {sub_item.get('label', 'N/A')}: {sub_item.get('score', 0):.2f}\n"
                          elif isinstance(item, dict):
                               formatted_result += f"- {item.get('label', 'N/A')}: {item.get('score', 0):.2f}\n"
                          else:
                               formatted_result += f"- {item}\n" # Fallback for other output types


                else:
                     formatted_result = "⚠️ تحلیل متن با مشکلی مواجه شد یا نتیجه‌ای در دسترس نیست."
            except Exception as analysis_model_error:
                 logger.error(f"Error during text analysis model processing: {analysis_model_error}")
                 formatted_result = "❌ خطایی در پردازش تحلیل متن توسط مدل رخ داد."


            increment_usage(user_id)
            user_data = get_user_data(user_id)
            remaining_uses = max(0, FREE_USAGE_LIMIT - user_data['usage_count'])

            response_text = f"{formatted_result}\n\n"
            if remaining_uses > 0:
                 response_text += f"🔢 شما {remaining_uses} استفاده رایگان دیگر دارید."
            else:
                 response_text += f"💡 سهمیه رایگان شما به پایان رسید. برای ادامه استفاده، لطفا {REFERRAL_REQUIRED} کاربر را معرفی کنید."


            await update.message.reply_text(response_text)

        except Exception as e:
            logger.error(f"General error handling text message for analysis: {e}")
            # Avoid sending a generic error if a more specific one was already generated
            if "خطایی در پردازش تحلیل متن توسط مدل رخ داد" not in formatted_result and \
               "⚠️ تحلیل متن با مشکلی مواجه شد" not in formatted_result:
                 await update.message.reply_text("❌ خطایی در پردازش درخواست تحلیل متن شما رخ داد.")


def main():
    """تابع اصلی برای راه‌اندازی ربات"""
    init_db()

    application = Application.builder().token(BOT_TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.VOICE & ~filters.COMMAND, handle_voice)) # Ensure it only handles voice messages that are not commands
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_text)) # Ensure it only handles text messages that are not commands

    print("✅ ربات در حال راه‌اندازی است...")
    # Added error handling for polling
    try:
        # Use asyncio.run to manage the event loop explicitly
        asyncio.run(application.run_polling())
    except Exception as e:
        logger.critical(f"Error during bot polling: {e}")
        print(f"❌ ربات با خطا متوقف شد: {e}")


if __name__ == "__main__":
    main()

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 186MiB/s]
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


model.safetensors:   0%|          | 0.00/473M [00:00<?, ?B/s]

Exception in thread Thread-4 (run_polling):
Traceback (most recent call last):
  File "/usr/lib/python3.12/asyncio/unix_events.py", line 105, in add_signal_handler
    signal.set_wakeup_fd(self._csock.fileno())
ValueError: set_wakeup_fd only works in main thread of the main interpreter

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_application.py", line 839, in run_polling
    return self.__run(
           ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/telegram/ext/_application.py", line 1044, in __run
    loop.add_signal_handler(sig, self._raise_system_exit)
  File "/usr/lib/python3.12/asyncio/unix_events.py", line 107, in add_signal_handler
    raise RuntimeErr

✅ ربات فعال شد! برای توقف، کلیدهای CTRL+C را فشار دهید.


/usr/lib/python3.12/threading.py:1077: RuntimeWarning: coroutine 'Updater.start_polling' was never awaited
  self._invoke_excepthook(self)


## انجام تست‌های جامع

### Subtask:
عملکرد ربات را به طور کامل تست کنید. این شامل ارسال پیام‌های صوتی، ارسال متون مختلف (حقوقی و غیرحقوقی)، بررسی پاسخ ربات به انتخاب‌های منو، و تست سناریوهای مربوط به عضویت در کانال و سهمیه استفاده است. نتایج تست را ثبت و هرگونه مشکل باقی‌مانده را شناسایی کنید.

In [ ]:
# %% [markdown]
# ## تنظیمات محیط و کلیدها
# لطفاً متغیرهای زیر را با مقادیر واقعی خود جایگزین کنید.

# %%
import os
from dotenv import load_dotenv

# تنظیم توکن ربات تلگرام شما
os.environ['TELEGRAM_BOT_TOKEN'] = '7693531934:AAEwCi1itefgAgdVZz_gvGLUc0DfbgkS6Tc'  # توکن خود را اینجا قرار دهید

# تنظیم کلید OpenAI برای دستیابی به بهترین عملکرد (اختیاری اما توصیه می‌شود)
os.environ['OPENAI_API_KEY'] = 'your-openai-api-key-here'  # اگر دارید، جایگزین کنید

# آدرس کانال اسپانسر (بات باید در کانال ادمین باشد)
os.environ['SPONSOR_CHANNEL'] = '@Radio_Zhelofen'

# تنظیمات مدل
model_size = "large"  # برای دقت بالاتر در تشخیص گفتار

In [ ]:
# Redundant installation cell, removing as dependencies are handled in the first cell
# pip install transformers==4.37.2

In [ ]:
pip show transformers

In [ ]:
from transformers import pipeline
print(pipeline('sentiment-analysis')('I love this!'))

In [ ]:
# Redundant installation cell, removing as dependencies are handled in the first cell
# pip install nest_asyncio
# import nest_asyncio
# nest_asyncio.apply()

# حالا کد اصلی ربات خود را اینجا اجرا کنید

In [ ]:
# -*- coding: utf-8 -*-
import logging
import os
import sqlite3
import asyncio
from datetime import datetime
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import (
    Application, CommandHandler, MessageHandler, filters,
    ConversationHandler, CallbackContext, CallbackQueryHandler
)
import whisper
import torch
from pytube import YouTube
import tempfile
import re

# تنظیمات لاگینگ
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# تنظیمات ربات
BOT_TOKEN = os.getenv('TELEGRAM_BOT_TOKEN', '7693531934:AAEwCi1itefgAgdVZz_gvGLUc0DfbgkS6Tc')
SPONSOR_CHANNEL = os.getenv('SPONSOR_CHANNEL', '@Radio_Zhelofen')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', 'your-openai-api-key-here')

# حالت های مکالمه
CHOOSING, TRACKING_REFERRALS = range(2)

# --- پایگاه داده برای مدیریت کاربران و دعوت‌ها ---
def init_db():
    conn = None
    try:
        conn = sqlite3.connect('users.db')
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS users
                     (user_id INTEGER PRIMARY KEY,
                      usage_count INTEGER DEFAULT 0,
                      referred_by INTEGER,
                      referral_count INTEGER DEFAULT 0)''')
        conn.commit()
        logger.info("Database initialized.")
    except sqlite3.Error as e:
        logger.error(f"Error initializing database: {e}")
    finally:
        if conn:
            conn.close()

def get_user_data(user_id):
    conn = None
    try:
        conn = sqlite3.connect('users.db')
        c = conn.cursor()
        c.execute("SELECT usage_count, referral_count FROM users WHERE user_id=?", (user_id,))
        result = c.fetchone()
        if result:
            return {'usage_count': result[0], 'referral_count': result[1]}
        return {'usage_count': 0, 'referral_count': 0}
    except sqlite3.Error as e:
        logger.error(f"Error retrieving user data for user {user_id}: {e}")
        return {'usage_count': 0, 'referral_count': 0} # Return default in case of error
    finally:
        if conn:
            conn.close()

def increment_usage(user_id):
    conn = None
    try:
        conn = sqlite3.connect('users.db')
        c = conn.cursor()
        c.execute('''INSERT OR REPLACE INTO users
                     (user_id, usage_count, referral_count)
                     VALUES (?, COALESCE((SELECT usage_count FROM users WHERE user_id=?), 0) + 1,
                             COALESCE((SELECT referral_count FROM users WHERE user_id=?), 0))''',
                  (user_id, user_id, user_id))
        conn.commit()
        logger.info(f"Usage count incremented for user {user_id}.")
    except sqlite3.Error as e:
        logger.error(f"Error incrementing usage for user {user_id}: {e}")
    finally:
        if conn:
            conn.close()


def add_referral(referrer_id):
    conn = None
    try:
        conn = sqlite3.connect('users.db')
        c = conn.cursor()
        c.execute("UPDATE users SET referral_count = referral_count + 1 WHERE user_id = ?", (referrer_id,))
        conn.commit()
        logger.info(f"Referral count incremented for user {referrer_id}.")
    except sqlite3.Error as e:
        logger.error(f"Error adding referral for user {referrer_id}: {e}")
    finally:
        if conn:
            conn.close()

# --- بررسی عضویت در کانال ---
async def check_channel_membership(user_id, context: CallbackContext):
    try:
        member = await context.bot.get_chat_member(chat_id=SPONSOR_CHANNEL, user_id=user_id)
        return member.status in ['member', 'administrator', 'creator']
    except Exception as e:
        logger.error(f"خطا در بررسی عضویت: {e}")
        return False

# --- مدیریت ویدیو و صوت ---
async def handle_media(update: Update, context: CallbackContext):
    user_id = update.effective_user.id

    try:
        # بررسی عضویت
        if not await check_channel_membership(user_id, context):
            await update.message.reply_text(
                f"⚠️ برای استفاده از ربات، باید در کانال اسپانسر عضو شوید:\n{SPONSOR_CHANNEL}\n"
                "پس از عضویت، مجدد اقدام کنید."
            )
            return

        user_data = get_user_data(user_id)
        if user_data['usage_count'] >= 5 and user_data['referral_count'] < 5:
            await update.message.reply_text(
                "❌ سهمیه رایگان شما به پایان رسید.\n"
                "برای ادامه استفاده، باید 5 کاربر را معرفی کنید.\n"
                "از منوی اصلی گزینه 'دعوت از دوستان' را انتخاب کنید."
            )
            return

        # پردازش رسانه
        await update.message.reply_text("🎥 در حال پردازش فایل ارسالی...")

        if update.message.video:
            file = await update.message.video.get_file()
            suffix = '.mp4' # Assuming mp4 for video, adjust if needed
        elif update.message.voice:
            file = await update.message.voice.get_file()
            suffix = '.ogg' # Assuming ogg for voice, adjust if needed
        else:
            await update.message.reply_text("⚠️ لطفا یک فایل ویدیویی یا صوتی ارسال کنید.")
            return

        # ذخیره فایل موقت
        file_path = None # Initialize to None
        try:
            with tempfile.NamedTemporaryFile(delete=False, suffix=suffix) as tmp_file:
                file_path = tmp_file.name
            await file.download_to_drive(file_path)

            # تبدیل به متن
            model = whisper.load_model("base") # Consider loading this once globally
            result = model.transcribe(file_path, language='fa')
            transcribed_text = result.get("text", "خطا در استخراج متن")

            # افزایش تعداد استفاده
            increment_usage(user_id)
            user_data = get_user_data(user_id) # Get updated data
            remaining_uses = 5 - (user_data['usage_count']) # Assuming 5 is the limit


            # ارسال نتیجه
            response_text = (
                f"📝 **متن استخراج شده:**\n\n{transcribed_text}\n\n"
                f"🔢 شما {max(0, 5 - user_data['usage_count'])} استفاده رایگان دیگر دارید." # Display remaining uses
            )
            await update.message.reply_text(response_text)

        except Exception as processing_error:
            logger.error(f"Error during media processing: {processing_error}")
            await update.message.reply_text("❌ خطایی در پردازش فایل شما رخ داد.")
        finally:
            # حذف فایل موقت
            if file_path and os.path.exists(file_path):
                try:
                    os.unlink(file_path)
                    logger.info(f"Temporary file deleted: {file_path}")
                except Exception as cleanup_error:
                    logger.error(f"Error deleting temporary file {file_path}: {cleanup_error}")


    except Exception as e:
        logger.error(f"General error in handle_media: {e}")
        await update.message.reply_text("❌ خطای کلی در پردازش درخواست شما رخ داد.")


# --- سیستم دعوت دوستان ---
async def share_invite_link(update: Update, context: CallbackContext):
    user_id = update.effective_user.id
    try:
        bot_username = (await context.bot.get_me()).username
        invite_link = f"https://t.me/{bot_username}?start=ref{user_id}"

        await update.message.reply_text(
            f"🔗 لینک دعوت اختصاصی شما:\n\n`{invite_link}`\n\n"
            "این لینک را برای دوستان خود ارسال کنید. پس از عضویت و استفاده آنان، "
            "سهمیه شما افزایش خواهد یافت.",
            parse_mode='Markdown'
        )
    except Exception as e:
        logger.error(f"Error sharing invite link: {e}")
        await update.message.reply_text("❌ خطایی در ایجاد لینک دعوت رخ داد.")


# --- مدیریت کاربران دعوت شده ---
async def track_referral(update: Update, context: CallbackContext):
    user_id = update.effective_user.id
    try:
        args = context.args

        if args and args[0].startswith('ref'):
            try:
                referrer_id = int(args[0][3:])
                if referrer_id != user_id:
                    add_referral(referrer_id)
                    await update.message.reply_text(
                        "✅ به ربات خوش آمدید! شما با لینک دعوت یکی از کاربران وارد شدید."
                    )
            except ValueError:
                logger.error(f"Invalid referrer ID format: {args[0]}")
                # Continue without adding referral if ID is invalid
            except Exception as e:
                logger.error(f"Error tracking referral: {e}")
                # Continue without adding referral in case of other errors

    except Exception as e:
        logger.error(f"General error in track_referral: {e}")
        # No message to user for this internal tracking function


# --- منوی اصلی ---
async def start(update: Update, context: CallbackContext):
    user_id = update.effective_user.id
    try:
        await track_referral(update, context) # Track referral on start

        if not await check_channel_membership(user_id, context):
            await update.message.reply_text(
                f"🔔 خوش آمدید!\n\n"
                f"برای استفاده از ربات هوشمند حقوقی، باید در کانال اسپانسر ما عضو شوید:\n"
                f"{SPONSOR_CHANNEL}\n\n"
                f"پس از عضویت، مجدد دستور /start را ارسال کنید."
            )
            return ConversationHandler.END

        user_data = get_user_data(user_id)
        keyboard = [
            [InlineKeyboardButton("🎥 آپلود ویدیو/صوت", callback_data="upload_media")],
            [InlineKeyboardButton("📝 تحلیل متن حقوقی", callback_data="analyze_text")],
            [InlineKeyboardButton("📄 تنظیم قرارداد", callback_data="generate_contract")],
            [InlineKeyboardButton("🔗 دعوت از دوستان", callback_data="invite_friends")],
            [InlineKeyboardButton("📊 وضعیت سهمیه", callback_data="usage_status")]
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)

        welcome_text = (
            "👋 به ربات هوشمند حقوقی خوش آمدید!\n\n"
            "⚖️ *امکانات ربات:*\n"
            "• تبدیل ویدیو و صوت به متن با دقت بالا\n"
            "• تحلیل تخصصی متون حقوقی\n"
            "• تنظیم قراردادهای هوشمند\n"
            "• پرسش و پاسخ حقوقی\n\n"
            "لطفاً یک گزینه را از منوی زیر انتخاب کنید:"
        )
        await update.message.reply_text(welcome_text, reply_markup=reply_markup, parse_mode='Markdown')
        return CHOOSING
    except Exception as e:
        logger.error(f"Error in start handler: {e}")
        await update.message.reply_text("❌ خطایی در شروع ربات رخ داد.")
        return ConversationHandler.END # End conversation on error


# --- مدیریت انتخاب های منو ---
async def handle_menu_selection(update: Update, context: CallbackContext):
    query = update.callback_query
    await query.answer()

    user_id = query.from_user.id
    user_data = get_user_data(user_id)

    try:
        if query.data == "upload_media":
            if user_data['usage_count'] >= 5 and user_data['referral_count'] < 5:
                await query.edit_message_text(
                    "❌ سهمیه رایگان شما به پایان رسید.\n\n"
                    "برای ادامه استفاده، باید ۵ کاربر را از طریق لینک دعوت خود به ربات معرفی کنید.\n\n"
                    "از گزینه 'دعوت از دوستان' در منو استفاده کنید."
                )
                return CHOOSING
            else:
                await query.edit_message_text(
                    "🎥 لطفاً ویدیو یا پیام صوتی خود را ارسال کنید.\n\n"
                    "✅ ربات از فرمت‌های مختلف پشتیبانی می‌کند.\n"
                    "⏱️ پردازش فایل ممکن است چند لحظه زمان ببرد."
                )
                return CHOOSING

        elif query.data == "invite_friends":
            bot_username = (await context.bot.get_me()).username
            invite_link = f"https://t.me/{bot_username}?start=ref{user_id}"

            await query.edit_message_text(
                f"👥 **سیستم دعوت از دوستان**\n\n"
                f"🔗 لینک اختصاصی شما:\n`{invite_link}`\n\n"
                f"📊 وضعیت فعلی:\n"
                f"• تعداد استفاده‌ها: {user_data['usage_count']}/5\n"
                f"• دوستان invited: {user_data['referral_count']}/5\n\n"
                f"✅ با هر ۵ دعوت موفق، ۵ استفاده رایگان جدید دریافت می‌کنید.",
                parse_mode='Markdown'
            )
            return CHOOSING

        elif query.data == "usage_status":
            await query.edit_message_text(
                f"📊 **وضعیت سهمیه شما**\n\n"
                f"• تعداد استفاده‌ها: {user_data['usage_count']}/5\n"
                f"• دوستان invited: {user_data['referral_count']}/5\n\n"
                f"🔁 پس از اتمام سهمیه، با دعوت از دوستان می‌توانید به استفاده ادامه دهید.",
                parse_mode='Markdown'
            )
            return CHOOSING

        else:
            await query.edit_message_text("🛠️ این قابلیت به زودی فعال خواهد شد.")
            return CHOOSING
    except Exception as e:
        logger.error(f"Error in handle_menu_selection: {e}")
        await query.edit_message_text("❌ خطایی در پردازش انتخاب شما رخ داد.")
        return CHOOSING # Stay in CHOOSING state


# --- تابع اصلی ---
def main():
    init_db()

    application = Application.builder().token(BOT_TOKEN).build()

    # هندلرهای مکالمه
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            CHOOSING: [
                CallbackQueryHandler(handle_menu_selection),
                MessageHandler(filters.VIDEO | filters.VOICE, handle_media)
            ]
        },
        fallbacks=[CommandHandler('start', start)]
    )

    application.add_handler(conv_handler)
    # The following line is redundant because handle_media is already handled in the ConversationHandler
    # application.add_handler(MessageHandler(filters.VIDEO | filters.VOICE, handle_media))

    print("✅ ربات فعال شد! برای توقف، کلیدهای CTRL+C را فشار دهید.")
    try:
        application.run_polling()
    except Exception as e:
        logger.critical(f"Error during bot polling: {e}")
        print(f"❌ ربات با خطا متوقف شد: {e}")


if __name__ == '__main__':
    main()

In [ ]:
!pip install --upgrade python-telegram-bot whisper-openai transformers torch sqlite3

In [ ]:
# -*- coding: utf-8 -*-
# نصب کتابخانه لازم (در صورت نبود)
# !pip install transformers torch torchaudio datasets

from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
# import torchaudio # Uncomment this line if you want to use torchaudio

# تعیین دستگاه پردازش (از GPU استفاده کن اگر در دسترس باشد)
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# بارگذاری پردازشگر و مدل large-v3
model_name = "openai/whisper-large-v3"
processor = WhisperProcessor.from_pretrained(model_name)
model = WhisperForConditionalGeneration.from_pretrained(model_name).to(device)

# اختیاری: برای تشخیص خودکار زبان، این شناسه‌ها را None بگذار
model.config.forced_decoder_ids = None

def transcribe_audio(audio_path):
    """
    تابع برای تبدیل صوت به متن
    """
    # بارگذاری فایل صوتی
    # TODO: Load the audio file from audio_path using a library like torchaudio or librosa.
    # Example with torchaudio:
    # waveform, sample_rate = torchaudio.load(audio_path)
    # audio_sample = {"array": waveform.squeeze().numpy(), "sampling_rate": sample_rate}

    # For now, using a placeholder for audio_sample
    # Replace this with actual audio loading
    audio_sample = None
    print(f"Placeholder: Loading audio from {audio_path}") # Placeholder print statement

    if audio_sample is None:
        print("Error: Audio sample not loaded. Please implement audio loading.")
        return "Error: Could not load audio."


    # پردازش ویژگی‌های ورودی
    input_features = processor(
        audio_sample["array"],
        sampling_rate=audio_sample["sampling_rate"],
        return_tensors="pt"
    ).input_features.to(device)

    # تولید شناسه‌های توکن
    predicted_ids = model.generate(input_features)

    # دیکد کردن به متن
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    return transcription[0]

# استفاده از تابع
if __name__ == "__main__":
    audio_file_path = "path/to/your/audio/file.wav"  # مسیر فایل صوتی خود را اینجا قرار دهید
    # TODO: Replace the placeholder audio_file_path with the actual path to your audio file.
    text_result = transcribe_audio(audio_file_path)
    print("متن استخراج شده:")
    print(text_result)

In [ ]:
# -*- coding: utf-8 -*-
import os
import logging
import sqlite3
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes
import whisper
import tempfile
from transformers import pipeline
import asyncio
import nest_asyncio # Import nest_asyncio

# Apply nest_asyncio to allow nested event loops, necessary for running asyncio in environments like Colab
nest_asyncio.apply()


# import torchaudio # Uncomment this line if you want to use torchaudio for audio loading

# تنظیمات لاگ‌گیری
logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO)
logger = logging.getLogger(__name__)

# توکن ربات و آیدی کانال — این مقادیر را جایگزین کرده‌اید
BOT_TOKEN = os.environ.get("TELEGRAM_BOT_TOKEN", "7693531934:AAEwCi1itefgAgdVZz_gvGLUc0DfbgkS6Tc") # Corrected variable name
SPONSOR_CHANNEL = os.environ.get("SPONSOR_CHANNEL", "@Radio_Zhelofen") # Corrected variable name

# --- پایگاه داده برای مدیریت کاربران و سهمیه ---
DB_NAME = 'users.db'

def init_db():
    conn = None
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        c.execute('''CREATE TABLE IF NOT EXISTS users
                     (user_id INTEGER PRIMARY KEY,
                      usage_count INTEGER DEFAULT 0,
                      referral_count INTEGER DEFAULT 0)''')
        conn.commit()
        logger.info("Database initialized.")
    except sqlite3.Error as e:
        logger.error(f"Error initializing database: {e}")
    finally:
        if conn:
            conn.close()


def get_user_data(user_id):
    conn = None
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        c.execute("SELECT usage_count, referral_count FROM users WHERE user_id=?", (user_id,))
        result = c.fetchone()
        if result:
            return {'usage_count': result[0], 'referral_count': result[1]}
        add_user(user_id) # Ensure user exists before returning default
        return {'usage_count': 0, 'referral_count': 0}
    except sqlite3.Error as e:
        logger.error(f"Error retrieving user data for user {user_id}: {e}")
        return {'usage_count': 0, 'referral_count': 0} # Return default in case of error
    finally:
        if conn:
            conn.close()

def add_user(user_id):
     conn = None
     try:
         conn = sqlite3.connect(DB_NAME)
         c = conn.cursor()
         c.execute("INSERT OR IGNORE INTO users (user_id, usage_count, referral_count) VALUES (?, 0, 0)", (user_id,))
         conn.commit()
         logger.info(f"User {user_id} added to DB if not exists.")
     except sqlite3.Error as e:
         logger.error(f"Error adding user {user_id} to DB: {e}")
     finally:
         if conn:
             conn.close()


def increment_usage(user_id):
    conn = None
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        c.execute("UPDATE users SET usage_count = usage_count + 1 WHERE user_id = ?", (user_id,))
        conn.commit()
        logger.info(f"Usage count incremented for user {user_id}.")
    except sqlite3.Error as e:
        logger.error(f"Error incrementing usage for user {user_id}: {e}")
    finally:
        if conn:
            conn.close()

def add_referral(referrer_id):
    conn = None
    try:
        conn = sqlite3.connect(DB_NAME)
        c = conn.cursor()
        c.execute("UPDATE users SET referral_count = referral_count + 1 WHERE user_id = ?", (referrer_id,))
        conn.commit()
        logger.info(f"Referral count incremented for user {referrer_id}.")
    except sqlite3.Error as e:
        logger.error(f"Error adding referral for user {referrer_id}: {e}")
    finally:
        if conn:
            conn.close()

# --- بررسی عضویت در کانال ---
async def check_channel_membership(user_id, context: ContextTypes.DEFAULT_TYPE):
    try:
        member = await context.bot.get_chat_member(chat_id=SPONSOR_CHANNEL, user_id=user_id)
        return member.status in ['member', 'administrator', 'creator']
    except Exception as e:
        logger.error(f"خطا در بررسی عضویت: {e}")
        return False


# Load the Whisper model once at the beginning
whisper_model = None
try:
    # Consider loading a smaller model if memory is an issue, e.g., "base.en" or "small"
    # Also consider specifying device="cuda" if a GPU is available and you have torch with CUDA
    whisper_model = whisper.load_model("base")
    logger.info("Whisper model loaded successfully.")
except Exception as e:
    logger.error(f"Error loading Whisper model: {e}")


# Load a text analysis model using Hugging Face pipeline
text_analyzer = None
try:
    # Using a general-purpose text classification model for Persian as a placeholder.
    # Finding a specific legal text analysis model for Persian in open-source is difficult.
    # This model is a general Persian text classification model. Its suitability for legal analysis is limited.
    # Replace with a more suitable model if one is found.
    text_analyzer = pipeline("text-classification", model="persiannlp/ParsBert-Political-Text-Classification") # Example Persian text classification model - replace if needed
    logger.info("Text analysis model loaded successfully.")
except Exception as e:
    logger.error(f"Error loading text analysis model: {e}")


async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """دستور /start — بررسی عضویت در کانال و نمایش منو"""
    user_id = update.effective_user.id

    add_user(user_id) # Ensure user exists for quota checks

    if not await check_channel_membership(user_id, context):
        await update.message.reply_text(
            f"⚠️ برای استفاده از ربات، لازم است ابتدا در کانال اسپانسر ما عضو شوید:\n\n"
            f"🌐 {SPONSOR_CHANNEL}\n\n"
            "پس از عضویت، مجدداً دستور /start را ارسال کنید."
        )
        return

    # If user is a member, show the main menu
    keyboard = [
        ["🎤 تبدیل صوت به متن", "📝 تحلیل حقوقی متن"],
        ["📄 ساخت قرارداد", "ℹ️ راهنما"]
    ]
    reply_markup = {"keyboard": keyboard, "resize_keyboard": True}
    await update.message.reply_text(
        "🔰 به ربات هوشمند حقوقی خوش آمدید!\n\n"
        "لطفاً یکی از گزینه‌های زیر را انتخاب کنید:",
        reply_markup=reply_markup
    )

async def handle_voice(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """مدیریت پیام‌های صوتی و تبدیل به متن"""
    user_id = update.effective_user.id

    # Check membership first
    if not await check_channel_membership(user_id, context):
         await update.message.reply_text(
            f"⚠️ برای استفاده از ربات، لازم است ابتدا در کانال اسپانسر ما عضو شوید:\n\n"
            f"🌐 {SPONSOR_CHANNEL}\n\n"
            "پس از عضویت، مجدداً دستور /start را ارسال کنید."
        )
         return

    # Then check usage quota
    user_data = get_user_data(user_id)
    FREE_USAGE_LIMIT = 5
    REFERRAL_REQUIRED = 5

    if user_data['usage_count'] >= FREE_USAGE_LIMIT and user_data['referral_count'] < REFERRAL_REQUIRED:
        await update.message.reply_text(
            f"❌ سهمیه رایگان {FREE_USAGE_LIMIT} تایی شما به پایان رسید.\n"
            f"برای ادامه استفاده، باید {REFERRAL_REQUIRED} کاربر را معرفی کنید.\n"
            "برای دریافت لینک دعوت خود، با مدیر تماس بگیرید." # Simplified message as referral link generation is not fully implemented
        )
        return

    if whisper_model is None:
        await update.message.reply_text("❌ متأسفانه مدل تبدیل صوت به متن بارگذاری نشده است. لطفاً به مدیر ربات اطلاع دهید.")
        return

    await update.message.reply_text("✅ پیام صوتی شما دریافت شد. در حال پردازش...")

    audio_path = None # Initialize audio_path to None
    try:
        voice_file = await update.message.voice.get_file()

        with tempfile.NamedTemporaryFile(delete=False, suffix='.ogg') as tmp_file:
            audio_path = tmp_file.name

        await voice_file.download_to_drive(audio_path)

        # Added error handling around transcription
        try:
            # Consider adding options like fp16=False if not using GPU or seeing issues
            result = whisper_model.transcribe(audio_path, language='fa')
            transcribed_text = result.get("text", "خطا در استخراج متن")
        except Exception as transcribe_error:
            logger.error(f"Error during Whisper transcription: {transcribe_error}")
            transcribed_text = "❌ خطایی در تبدیل پیام صوتی به متن رخ داد."


        increment_usage(user_id)
        user_data = get_user_data(user_id)
        remaining_uses = max(0, FREE_USAGE_LIMIT - user_data['usage_count'])

        response_text = f"📝 متن استخراج شده:\n\n{transcribed_text}\n\n"
        if remaining_uses > 0:
             response_text += f"🔢 شما {remaining_uses} استفاده رایگان دیگر دارید."
        else:
             response_text += f"💡 سهمیه رایگان شما به پایان رسید. برای ادامه استفاده، لطفا {REFERRAL_REQUIRED} کاربر را معرفی کنید."

        await update.message.reply_text(response_text)

    except Exception as e:
        logger.error(f"Error handling voice message: {e}")
        await update.message.reply_text("❌ خطایی در پردازش پیام صوتی شما رخ داد.")
    finally:
        # Ensure the temporary file is deleted
        if audio_path and os.path.exists(audio_path):
            try:
                os.unlink(audio_path)
                logger.info(f"Temporary file deleted: {audio_path}")
            except Exception as cleanup_error:
                logger.error(f"Error deleting temporary file {audio_path}: {cleanup_error}")



async def handle_text(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """مدیریت پیام‌های متنی و تحلیل حقوقی"""
    user_id = update.effective_user.id
    user_text = update.message.text

    # Check membership first
    if not await check_channel_membership(user_id, context):
         await update.message.reply_text(
            f"⚠️ برای استفاده از ربات، لازم است ابتدا در کانال اسپانسر ما عضو شوید:\n\n"
            f"🌐 {SPONSOR_CHANNEL}\n\n"
            "پس از عضویت، مجدداً دستور /start را ارسال کنید."
        )
         return


    if user_text == "🎤 تبدیل صوت به متن":
        await update.message.reply_text("لطفاً پیام صوتی خود را ارسال کنید.")
    elif user_text == "📝 تحلیل حقوقی متن":
        await update.message.reply_text("لطفاً متن حقوقی خود را برای تحلیل ارسال کنید.")
    elif user_text == "📄 ساخت قرارداد":
        await update.message.reply_text("این قابلیت به زودی فعال خواهد شد.")
    elif user_text == "ℹ️ راهنما":
         await update.message.reply_text(
             "📚 **راهنمای استفاده از ربات:**\n\n"
             "• برای تبدیل صوت به متن، گزینه '🎤 تبدیل صوت به متن' را انتخاب کرده و سپس پیام صوتی خود را ارسال کنید.\n"
             "• برای تحلیل متن حقوقی، گزینه '📝 تحلیل حقوقی متن' را انتخاب کرده و سپس متن مورد نظر را ارسال کنید.\n"
             "• قابلیت '📄 ساخت قرارداد' به زودی اضافه خواهد شد.\n"
             "• برای بازگشت به منوی اصلی، دستور /start را ارسال کنید."
         )
    elif text_analyzer is None:
        await update.message.reply_text("❌ متأسفانه مدل تحلیل متن بارگذاری نشده است. لطفاً به مدیر ربات اطلاع دهید.")
    else:
        # Apply usage quota check for text analysis
        user_data = get_user_data(user_id)
        FREE_USAGE_LIMIT = 5
        REFERRAL_REQUIRED = 5

        if user_data['usage_count'] >= FREE_USAGE_LIMIT and user_data['referral_count'] < REFERRAL_REQUIRED:
            await update.message.reply_text(
                f"❌ سهمیه رایگان {FREE_USAGE_LIMIT} تایی شما به پایان رسید.\n"
                f"برای ادامه استفاده، باید {REFERRAL_REQUIRED} کاربر را معرفی کنید.\n"
                "برای دریافت لینک دعوت خود، با مدیر تماس بگیرید."
            )
            return


        await update.message.reply_text("✅ متن شما دریافت شد. در حال تحلیل حقوقی...")
        formatted_result = "⚠️ تحلیل متن با مشکلی مواجه شد یا نتیجه‌ای در دسترس نیست." # Initialize formatted_result
        try:
            # Added error handling around text analysis
            try:
                analysis_result = text_analyzer(user_text)
                if analysis_result and isinstance(analysis_result, list) and len(analysis_result) > 0:
                     formatted_result = f"📊 نتیجه تحلیل:\n\n"
                     # The output format depends on the specific model.
                     # For a classification model, it might be a list of labels and scores.
                     # Adjust this formatting based on the actual output of the chosen Persian model.
                     # Example formatting assuming a list of dicts with 'label' and 'score':
                     for item in analysis_result:
                          # Need to handle potential list of lists or other structures depending on the model
                          if isinstance(item, list):
                              for sub_item in item:
                                   formatted_result += f"- {sub_item.get('label', 'N/A')}: {sub_item.get('score', 0):.2f}\n"
                          elif isinstance(item, dict):
                               formatted_result += f"- {item.get('label', 'N/A')}: {item.get('score', 0):.2f}\n"
                          else:
                               formatted_result += f"- {item}\n" # Fallback for other output types


                else:
                     formatted_result = "⚠️ تحلیل متن با مشکلی مواجه شد یا نتیجه‌ای در دسترس نیست."
            except Exception as analysis_model_error:
                 logger.error(f"Error during text analysis model processing: {analysis_model_error}")
                 formatted_result = "❌ خطایی در پردازش تحلیل متن توسط مدل رخ داد."


            increment_usage(user_id)
            user_data = get_user_data(user_id)
            remaining_uses = max(0, FREE_USAGE_LIMIT - user_data['usage_count'])

            response_text = f"{formatted_result}\n\n"
            if remaining_uses > 0:
                 response_text += f"🔢 شما {remaining_uses} استفاده رایگان دیگر دارید."
            else:
                 response_text += f"💡 سهمیه رایگان شما به پایان رسید. برای ادامه استفاده، لطفا {REFERRAL_REQUIRED} کاربر را معرفی کنید."


            await update.message.reply_text(response_text)

        except Exception as e:
            logger.error(f"General error handling text message for analysis: {e}")
            # Avoid sending a generic error if a more specific one was already generated
            if "خطایی در پردازش تحلیل متن توسط مدل رخ داد" not in formatted_result and \
               "⚠️ تحلیل متن با مشکلی مواجه شد" not in formatted_result:
                 await update.message.reply_text("❌ خطایی در پردازش درخواست تحلیل متن شما رخ داد.")


def main():
    """تابع اصلی برای راه‌اندازی ربات"""
    init_db()

    application = Application.builder().token(BOT_TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.VOICE & ~filters.COMMAND, handle_voice)) # Ensure it only handles voice messages that are not commands
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_text)) # Ensure it only handles text messages that are not commands

    print("✅ ربات فعال شد! برای توقف، کلیدهای CTRL+C را فشار دهید.")
    # Added error handling for polling
    try:
        application.run_polling()
    except Exception as e:
        logger.critical(f"Error during bot polling: {e}")
        print(f"❌ ربات با خطا متوقف شد: {e}")


if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'telegram'

In [ ]:
# تغییر از مدل base به large
model = whisper.load_model("large", device="cuda")  # استفاده از GPU اگر در دسترس است

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
# -*- coding: utf-8 -*-
import os
import logging
from telegram import Update
from telegram.ext import Application, MessageHandler, filters, ContextTypes

# تنظیمات لاگ‌گیری
logging.basicConfig(format="%(asctime)s - %(name)s - %(levelname)s - %(message)s", level=logging.INFO)
logger = logging.getLogger(__name__)

# توکن ربات و آیدی کانال — این مقادیر را جایگزین کرده‌اید
BOT_TOKEN = os.environ.get("BOT_TOKEN", "7693531934:AAEwCi1itefgAgdVZz_gvGLUc0DfbgkS6Tc")
SPONSOR_CHANNEL = "@Radio_Zhelofen"  # آیدی کانال اسپانسر

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """دستور /start — بررسی عضویت در کانال و نمایش منو"""
    user_id = update.effective_user.id

    # بررسی عضویت کاربر در کانال اسپانسر
    try:
        member = await context.bot.get_chat_member(chat_id=SPONSOR_CHANNEL, user_id=user_id)
        if member.status not in ["member", "administrator", "creator"]:
            await update.message.reply_text(
                f"⚠️ برای استفاده از ربات، لازم است ابتدا در کانال اسپانسر ما عضو شوید:\n\n"
                f"🌐 {SPONSOR_CHANNEL}\n\n"
                "پس از عضویت، مجدداً دستور /start را ارسال کنید."
            )
            return
    except Exception as e:
        logger.error(f"خطا در بررسی عضویت کاربر در کانال: {e}")
        await update.message.reply_text("❌ خطایی در بررسی عضویت رخ داد. لطفاً بعداً تلاش کنید.")
        return

    # اگر کاربر عضو کانال باشد، منوی اصلی نمایش داده می‌شود
    keyboard = [
        ["🎤 تبدیل صوت به متن", "📝 تحلیل حقوقی متن"],
        ["📄 ساخت قرارداد", "ℹ️ راهنما"]
    ]
    reply_markup = {"keyboard": keyboard, "resize_keyboard": True}
    await update.message.reply_text(
        "🔰 به ربات هوشمند حقوقی خوش آمدید!\n\n"
        "لطفاً یکی از گزینه‌های زیر را انتخاب کنید:",
        reply_markup=reply_markup
    )

async def handle_voice(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """مدیریت پیام‌های صوتی و تبدیل به متن"""
    # در این بخش باید کد مربوط به مدل متن‌باز تبدیل صوت به متن (مثلاً Whisper) فراخوانی شود.
    await update.message.reply_text("✅ پیام صوتی شما دریافت شد. در حال پردازش...")

    # TODO: این بخش با کتابخانه‌های متن‌باز مانند Whisper تکمیل شود
    # voice_file = await update.message.voice.get_file()
    # transcribed_text = your_whisper_function(voice_file.file_path)

    transcribed_text = "متن نمونه استخراج شده از صوت"  # این یک متن نمونه است

    await update.message.reply_text(f"📝 متن استخراج شده:\n\n{transcribed_text}")

async def handle_text(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """مدیریت پیام‌های متنی و تحلیل حقوقی"""
    user_text = update.message.text

    if user_text == "🎤 تبدیل صوت به متن":
        await update.message.reply_text("لطفاً پیام صوتی خود را ارسال کنید.")
    elif user_text == "📝 تحلیل حقوقی متن":
        await update.message.reply_text("لطفاً متن حقوقی خود را برای تحلیل ارسال کنید.")
    elif user_text == "📄 ساخت قرارداد":
        await update.message.reply_text("این قابلیت به زودی فعال خواهد شد.")
    else:
        # TODO: این بخش با مدل زبانی متن‌باز (مثلاً از Hugging Face) برای تحلیل متن تکمیل شود
        await update.message.reply_text("درخواست شما دریافت شد. (تحلیل توسط هوش مصنوعی)")

def main():
    """تابع اصلی برای راه‌اندازی ربات"""
    application = Application.builder().token(BOT_TOKEN).build()

    # ثبت هندلرها
    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.VOICE, handle_voice))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_text))

    # شروع ربات
    application.run_polling()
    print("ربات فعال شد!")

if __name__ == "__main__":
    main()

RuntimeError: Cannot close a running event loop

In [ ]:
async def handle_media(update: Update, context: CallbackContext):
    try:
        # کد اصلی شما
    except Exception as e:
        logger.error(f"خطا در پردازش رسانه: {e}")
        await update.message.reply_text("⚠️ متأسفانه در پردازش فایل مشکلی پیش آمد.")

IndentationError: expected an indented block after 'try' statement on line 2 (ipython-input-694587228.py, line 4)

# Task
Debug the provided Python code for a Telegram bot, specifically addressing issues related to voice-to-text conversion errors, non-functional features, and general bugs. The debugging process should involve examining logs, completing incomplete code sections (voice-to-text and text analysis), reviewing message handling logic, checking membership/quota systems (if applicable), adding error handling, and performing comprehensive testing. The final response should be in Persian, explaining the root cause of the errors and why other features are not working.

## بررسی لاگ‌ها و خطاهای موجود

### Subtask:
لاگ‌های خروجی سلول‌های قبلی، به خصوص `vHGwyym0ML0d` و `mq3gyxg2MiKW` را بررسی کنید تا خطاهای زمان اجرا یا هشدارهای موجود را شناسایی کنید.


## تکمیل قابلیت تبدیل صوت به متن

### Subtask:
کد مربوط به استفاده از مدل Whisper (یا مدل جایگزین) برای تبدیل فایل صوتی دریافتی از کاربر به متن را در تابع `handle_voice` پیاده‌سازی کنید.


**Reasoning**:
Implement the voice-to-text conversion in the `handle_voice` function by loading the Whisper model, downloading the audio file, transcribing it, and sending the result to the user, ensuring temporary files are handled properly.



## تکمیل قابلیت تحلیل متن حقوقی

### Subtask:
کد مربوط به استفاده از یک مدل زبانی (مانند مدل‌های Hugging Face) برای تحلیل متن حقوقی ارسالی توسط کاربر را در تابع `handle_text` پیاده‌سازی کنید.


**Reasoning**:
Implement the text analysis feature using a Hugging Face model within the `handle_text` function. This involves loading a suitable model and tokenizer, processing the user's text, and sending the result back.



## بررسی و اصلاح منطق مدیریت پیام‌ها

### Subtask:
اطمینان حاصل کنید که `MessageHandler` ها و `CommandHandler` ها به درستی تنظیم شده‌اند و پیام‌های مختلف (دستور `start`، پیام صوتی، پیام متنی) به توابع صحیح هدایت می‌شوند. منطق مربوط به منو و پاسخ به انتخاب‌های کاربر را بازبینی کنید.


**Reasoning**:
بررسی می‌کنم که هندلرهای پیام به درستی تعریف و به application اضافه شده‌اند و منطق مدیریت پیام‌های متنی در تابع `handle_text` صحیح است.



## بررسی منطق سیستم عضویت و سهمیه (در صورت استفاده)

### Subtask:
اگر از سیستم بررسی عضویت در کانال یا سهمیه استفاده می‌کنید (مانند آنچه در سلول `vHGwyym0ML0d` بود)، منطق آن را بررسی کنید و مطمئن شوید که به درستی کار می‌کند و در مکان‌های مناسب (مثلاً قبل از ارائه خدمات اصلی) بررسی می‌شود.


**Reasoning**:
بررسی می‌کنم که آیا منطق بررسی عضویت در کانال و سیستم سهمیه در کد فعلی (سلول `mq3gyxg2MiKW`) وجود دارد یا خیر. اگر وجود نداشته باشد، تصمیم می‌گیرم که آیا آن را اضافه کنم یا خیر و در صورت لزوم منطق آن را اضافه می‌کنم.



**Reasoning**:
منطق بررسی عضویت در کانال و سیستم سهمیه به کد اضافه شده است. حالا باید اطمینان حاصل کنم که این منطق به درستی پیاده‌سازی شده و در مکان‌های صحیح (یعنی قبل از ارائه خدمات اصلی) بررسی می‌شود و پیام‌های مناسب به کاربر نمایش داده می‌شود.



In [ ]:
# Run the bot code cell
# Removed get_ipython().run_cell('96128783') to avoid running main multiple times

In [ ]:
!pip install python-telegram-bot

## افزودن مدیریت خطا

### Subtask:
افزودن مدیریت خطا برای جلوگیری از از کار افتادن ربات در صورت بروز مشکلات پیش‌بینی نشده در طول اجرای توابع مختلف.


**Reasoning**:
Add try...except blocks to the relevant functions and model loading sections to handle potential errors, log them, and send user-friendly messages, fulfilling the error handling requirement of the subtask.

